In [ ]:
!wget https://huggingface.co/danjacobellis/dance/resolve/main/video_f8c48.pth

In [1]:
import torch
import numpy as np
from autocodec.codec import AutoCodecND

In [2]:
checkpoint = torch.load('video_f8c48.pth', map_location="cpu",weights_only=False)
config = checkpoint['config']
state_dict = checkpoint['state_dict']
model = AutoCodecND(
    dim=3,
    input_channels=config.input_channels,
    J = int(np.log2(config.F)),
    latent_dim=config.latent_dim,
    encoder_depth = config.encoder_depth,
    encoder_kernel_size = config.encoder_kernel_size,
    decoder_depth = config.decoder_depth,
    lightweight_encode = config.lightweight_encode,
    lightweight_decode = config.lightweight_decode,
)
model.load_state_dict(state_dict)
del model.decoder_blocks

In [3]:
sum(p.numel() for p in model.parameters())/1e6

1.417008

In [4]:
torch.save({
    'config': config,
    'state_dict': model.state_dict()
}, '../../hf/dance/video_f8c48_encoder.pth')

In [1]:
import torch
import datasets

In [2]:
ds = datasets.load_dataset("danjacobellis/davis3",split='validation').cast_column('video', datasets.Video()).with_format("torch")

In [3]:
for sample in ds:
    video = torch.cat([f['data'].unsqueeze(1) for f in sample['video']],dim=1).unsqueeze(0)

In [4]:
video.shape

torch.Size([1, 3, 25, 1080, 1920])